In [1]:

import random
import os
import autogen
import openai

In [10]:
openai.api_type = "azure"
openai.api_base = os.environ.get("AZURE_OPENAI_API_BASE", "https://airt-openai-sweden.openai.azure.com/")
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai.api_version = "2023-07-01-preview"

config_list = [
    {
        "model": "gpt-4",
#         "model": "gpt-3.5-turbo-16k",
        "api_key": openai.api_key,
        "api_base": openai.api_base,
        "api_type": openai.api_type,
        "api_version": openai.api_version,
        "engine": "airt-gpt4"
#         "engine": "airt-gpt35-turbo-16k"
    }
]

In [3]:
SYSTEM_MESSAGE_PRODUCT_OWNER = """
You are a product owner which works with streaming application projects (using Kafka) and all the applications are using python FastStream framework.
You are in a chat with the client and team leader to discuss the requirements for the new application that needs to be implemented.
Go through the given application description, and check if everything what you think is necessary is defined.

Your response should always be ONE of the following: 
1. If you have a question for the user or team leader please ask it. Please ask ONE QUESTION at time.
2. Before you create a description needed for writing a technical brief by calling the "create_brief", please list all assumptions and information and then verify with the user that you understanding is correct.

Suggest calling "create_brief" function when everything is clear to you have no more questions.

If "create_bried" returns with additional questions, please include the additional answers and clarifications in a new suggestion for calling the "create_brief" again.

Once the "create_brief" function returns the technical brief, check if it correct and in line with the user requests. If you notice any discrepancies,
please raise you concerns immediately and suggest calling the "create_brief" again with more elaborate instructions.

"""
# 

In [4]:
create_brief = {
            "name": "create_brief",
            "description": "Create brief from global variable",
            "parameters": {
                "type": "object",
                "properties": {
                    "desc": {
                        "type": "string",
                        "description": "Description of the software as desired by the user with all additional clarificatioins and information.",
                    },
                },
                "required": ["desc"],
            }
        }


llm_config_product_owner = {
    "config_list": config_list,
    "seed": 42,
    "temperature": 0.7,  # temperature for sampling
    "functions": [create_brief],
} 

# user_proxy = autogen.UserProxyAgent(
#    name="User_proxy",
#    system_message="A human admin.",
#    human_input_mode="AUTO"
# )
user_proxy = autogen.AssistantAgent(
   name="User_proxy",
   system_message="""A non-technical user wishing to outsource software development work to a software agency.
   
You are interested in Bitcoing and Eth cryptocurrency and have no interest for others. 

Since you know next to nothing about the technology, you will leave all technical decisions to the agency.

Make your answers as short as possible, i.e. yes/no/don't care.

You will terminate the discussion by outputing the word TERMINATE once you are satisfied with the output.
""",
   llm_config=llm_config_product_owner,
)

product_owner = autogen.AssistantAgent(
    name="Product_owner",
    system_message=SYSTEM_MESSAGE_PRODUCT_OWNER,
    llm_config=llm_config_product_owner,
)

In [5]:
called_already: bool = False
def create_brief(desc: str):
    print(f"create_brief({desc=})")
    global called_already
    if called_already:
        return """
    Create a FastStream application which will retrieve the current cryptocurrency price
    and publish it to new_crypto_price topic. 

    The application should retrieve the data every 2 seconds.

    A message which will be produced is JSON with the two attributes:
    - price: non-negative float (current price of cryptocurrency in USD)
    - crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)

    Use utf-8 encoded crypto_currency attribute as a partition key when publishing
    the message to new_crypto_price topic.

    To retrieve the prices, use the CryptoCompare API.
    Retrieve only Bitcoin and ETH prices.
"""
    called_already = True
    
    return "I am not clear on what API should I use for retrieving crypto prises. I recommend Coinbase API, is it fine with you?"
        

# function_executer_proxy = autogen.UserProxyAgent(
#     name="function_executer_proxy",
#     human_input_mode="NEVER",
#     max_consecutive_auto_reply=10,
#     # is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
#     code_execution_config={"work_dir": "planning"},
#     function_map={"create_brief": create_brief},
# )

In [6]:
function_executor_proxy = autogen.UserProxyAgent(
    name="Function_executor_proxy",
    #    system_message="A human admin.",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "planning"},
    function_map={"create_brief": create_brief},
)

In [7]:
description = """
Create a FastStream application which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

The application should retrieve the data every 2 seconds.

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)

Use utf-8 encoded crypto_currency attribute as a partition key when publishing
the message to new_crypto_price topic.

"""

initial_message = f"""{description}

Is everything clear with the above description?
"""

llm_config = {
    "config_list": config_list,
    "seed": 42,
    "temperature": 0.7,  # temperature for sampling
} 

groupchat = autogen.GroupChat(agents=[user_proxy, product_owner, function_executor_proxy], messages=[], max_round=20)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

user_proxy.initiate_chat(manager, message=initial_message)
# user_proxy.initiate_chat(product_owner, message=initial_message)

User_proxy (to chat_manager):


Create a FastStream application which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

The application should retrieve the data every 2 seconds.

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)

Use utf-8 encoded crypto_currency attribute as a partition key when publishing
the message to new_crypto_price topic.



Is everything clear with the above description?


--------------------------------------------------------------------------------
Product_owner (to chat_manager):

I have a few questions to clarify the requirements further:

1. From which source or API should the application retrieve the cryptocurrency price?
2. Should the application support multiple cryptocurrencies or only one specific cryptocurrency?

-------------------------------------------------

In [9]:
assert False

AssertionError: 

In [ ]:
for message in reversed(product_owner.chat_messages[user_proxy]):
    if 'function_call' in message:
#         print(message["function_call"])
        create_brief_result = locals()[message["function_call"]["name"]]()
        print(create_brief_result)
        break

## Continue the conversation

In [ ]:
message = "Updated application description:\n" + create_brief_result + "\nIf you agree with the application description above, type 'exit'?"
product_owner.send(recipient=user_proxy, message=message)

In [ ]:
for message in product_owner.chat_messages[user_proxy]:
    role_message = f"{message['role']}:\n{message['content']}"
    print(role_message)